In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import ast
import torch
import pickle
import keyboard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split

from marl_aquarium import aquarium_v0

from utils.env_utils import *
from utils.eval_utils import *
from utils.train_utils import *
from utils.es_utils import *

from models.Buffer import Pool, Buffer
from models.DeepGenerator import GeneratorPolicy
from models.Discriminator import Discriminator

In [2]:
'''
ToDo's
Descriptive Beschreibung des Ergebnisse
    - 

Benchmark-Verlgeich
    - Vergleich des Modells mit Expert Data, Random Model, Viscek/Couzin
    - benötige ich Viscek/Couzin wenn ich Expert Data habe?
    - welche Metriken? Polarisation etc. nicht geeignet da keine Performance Metriken - Informationsfluss in Schwarm?

Rollenverteilung im Schwarm (Garcia):
    - Nach Papern zu Rollen in Fischschwärmen suchen
    - Rollenverteilung im Schwarm analysieren (Führungsfisch, ...)
    - Wie variieren bestimmte Rollen? Nach Distanz, ...
    - Attention-Netz nutzen um Distances zu analysieren

Trajectory Prediction under Survival Pressure (Liang):
    - Vorhersage von Fisch-Trajektorien mit den drei States "ohne Predator", "mit Predator", "Angriff Predator" (Monte-Carlo simulieren von bspw. 25-frames Traj durch Model mit realem Startzustand)
    - Descriptive Beschreibung wie sich Schwarm durch Präsenz des Predators ändert (Polarization, Angular Momentum, Density, ...)
    - Korrelationsberechnungen zwischen Schwarmmetriken und bspw. Predator-Distanz - Distanz zwischen Fischen nutzen wie im Parisi Paper

Zerfall von Ordnung zu Chaos (Hamann):
    - Analyse des "criticial points" im Schwarm (Berechnung der Differenz in Schwimmrichtung)
    - Analyse des Fountain Effects 
    - Vehicle-Crows-Interaction (Yang et al.), Vergleich Fahrzeug in Menschenmenge mit Predator in Fischschwarm
'''

'\nToDo\'s\nDescriptive Beschreibung des Ergebnisse\n    - \n\nBenchmark-Verlgeich\n    - Vergleich des Modells mit Expert Data, Random Model, Viscek/Couzin\n    - benötige ich Viscek/Couzin wenn ich Expert Data habe?\n    - welche Metriken? Polarisation etc. nicht geeignet da keine Performance Metriken - Informationsfluss in Schwarm?\n\nRollenverteilung im Schwarm (Garcia):\n    - Nach Papern zu Rollen in Fischschwärmen suchen\n    - Rollenverteilung im Schwarm analysieren (Führungsfisch, ...)\n    - Wie variieren bestimmte Rollen? Nach Distanz, ...\n    - Attention-Netz nutzen um Distances zu analysieren\n\nTrajectory Prediction under Survival Pressure (Liang):\n    - Vorhersage von Fisch-Trajektorien mit den drei States "ohne Predator", "mit Predator", "Angriff Predator" (Monte-Carlo simulieren von bspw. 25-frames Traj durch Model mit realem Startzustand)\n    - Descriptive Beschreibung wie sich Schwarm durch Präsenz des Predators ändert (Polarization, Angular Momentum, Density, ...

In [3]:
# Training Folder
total_detections = 33
policy_dir = rf"..\data\pretuning\Pretuning - 25.09.2025_17.25"
data_path = rf'..\data\processed\pred_prey_interactions\expert_tensors\{total_detections}'
hl_path = rf'..\data\processed\pred_prey_interactions\expert_tensors\hand_labeled'
ftw_path = rf'..\data\processed\pred_prey_interactions\full_track_windows\{total_detections}'

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
expert_buffer = Buffer(pred_max_length=23000, prey_max_length=73000, device=device)

# Load Expert Data from local storage
print("Expert Buffer is empty, load data...")
expert_buffer.add_expert(data_path)
expert_buffer.clear(p=80)               # Reduce ratio of non-attack data by 90%. now ~equal
expert_buffer.add_expert(hl_path)       # hand-labeled data | Pred: 1057 | Prey: 33824
expert_buffer.add_expert(hl_path)
len_exp_pred, len_exp_prey = expert_buffer.lengths()

print("Storage of Predator Expert Buffer: ", len_exp_pred)
print("Storage of Prey Expert Buffer: ", len_exp_prey, "\n")

Expert Buffer is empty, load data...
Storage of Predator Expert Buffer:  6398
Storage of Prey Expert Buffer:  73000 



In [5]:
pred_policy = torch.load(os.path.join(policy_dir, "pred_policy.pt"), weights_only=False)
prey_policy = torch.load(os.path.join(policy_dir, "prey_policy.pt"), weights_only=False)

In [6]:
# Expert Data
total_detections=33
ftw_path = rf"..\data\processed\pred_prey_interactions\full_track_windows\{total_detections}"

start_frame_pool = Pool(max_length=12100, device=device)
start_frame_pool.generate_startframes(ftw_path)
print(f"Start frames in pool: {len(start_frame_pool)}")

Start frames in pool: 11978


In [8]:
env = parallel_env(predator_count=1, prey_count=32, action_count=360, use_walls=False)
positions = start_frame_pool.sample(n=1)
env.reset(options=positions)
metrics = run_policies(env, pred_policy, prey_policy, prey_features=5)

Press 'q' to end simulation.


In [ ]:
'''
Unterteilung in Leader, Follower, Initiators (und auch Recruiters welche andere animieren)
- Leadership sei teilweise asymmetrisch
- Geschwindigkeits-induzierte Führung (https://doi.org/10.1016/j.cub.2017.08.004)
- Verhaltenskaskaden beim Informationsfluss (https://doi.org/10.1038/s41598-022-14337-0)
'''